<a href="https://www.kaggle.com/code/mrsimple07/lightgbm-with-the-score-0-9112?scriptVersionId=162740599" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


# Importing data and libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report,roc_auc_score,accuracy_score
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler,OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import xgboost as xgb
import optuna

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s4e2/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e2/test.csv')
sample_submission = pd.read_csv('/kaggle/input/playground-series-s4e2/sample_submission.csv')

# EDA

In [4]:
train.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [5]:
test.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
1,20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
2,20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
3,20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
4,20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation


In [6]:
sample_submission.head()

,id,NObeyesdad
0,20758,Normal_Weight
1,20759,Normal_Weight
2,20760,Normal_Weight
3,20761,Normal_Weight
4,20762,Normal_Weight


# Model selection & Evaluation

In [7]:
categorical_columns = ["Gender", "family_history_with_overweight", "FAVC", "CAEC", "SMOKE", "SCC", "CALC", "MTRANS"]

# Convert categorical columns using your chosen method
train[categorical_columns] = train[categorical_columns].apply(pd.Categorical)
test[categorical_columns] = test[categorical_columns].apply(pd.Categorical)

In [8]:
train = train.drop(['id'],axis=1).drop_duplicates()
test = test.drop(['id'], axis=1)

In [9]:
X = train.drop(['NObeyesdad'], axis=1)
y= train['NObeyesdad']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
import warnings 

warnings.filterwarnings('ignore')

def objective(trial, X_train, y_train, X_test, y_test):
#     # Define parameters to be optimized for the LGBMClassifier
    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "random_state": 42,
        "num_class": 7,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.05),
        "n_estimators": trial.suggest_int("n_estimators", 400, 600),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.005, 0.015),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.02, 0.06),
        "max_depth": trial.suggest_int("max_depth", 6, 14),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 0.9),
        "subsample": trial.suggest_float("subsample", 0.8, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
    }

#     # Create an instance of LGBMClassifier with the suggested parameters
    lgbm_classifier = LGBMClassifier(**param)
    
#     # Fit the classifier on the training data
    lgbm_classifier.fit(X_train, y_train)

#     # Evaluate the classifier on the test data
    score = lgbm_classifier.score(X_test, y_test)

    return score

# # Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Adjust the test_size as needed

# # Set up the sampler for Optuna optimization
sampler = optuna.samplers.TPESampler(seed=42)  # Using Tree-structured Parzen Estimator sampler for optimization

# # Create a study object for Optuna optimization
study = optuna.create_study(direction="maximize", sampler=sampler)

# # Run the optimization process
study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=100)

# # Get the best parameters after optimization
best_params = study.best_params

print('='*50)
print(best_params)

[I 2024-02-13 13:44:26,600] A new study created in memory with name: no-name-4c0f7a08-b30e-4017-ba74-9559bfc5c3af
[I 2024-02-13 13:44:47,712] Trial 0 finished with value: 0.9051059730250481 and parameters: {'learning_rate': 0.0249816047538945, 'n_estimators': 591, 'lambda_l1': 0.012319939418114049, 'lambda_l2': 0.04394633936788146, 'max_depth': 7, 'colsample_bytree': 0.3935967122017216, 'subsample': 0.8116167224336399, 'min_child_samples': 45}. Best is trial 0 with value: 0.9051059730250481.
[I 2024-02-13 13:45:01,263] Trial 1 finished with value: 0.9060693641618497 and parameters: {'learning_rate': 0.034044600469728355, 'n_estimators': 542, 'lambda_l1': 0.005205844942958024, 'lambda_l2': 0.05879639408647977, 'max_depth': 13, 'colsample_bytree': 0.4274034664069657, 'subsample': 0.8363649934414201, 'min_child_samples': 17}. Best is trial 1 with value: 0.9060693641618497.
[I 2024-02-13 13:45:15,142] Trial 2 finished with value: 0.9058285163776493 and parameters: {'learning_rate': 0.02216

{'learning_rate': 0.015797659065062593, 'n_estimators': 425, 'lambda_l1': 0.008724491316001835, 'lambda_l2': 0.020209622675945973, 'max_depth': 10, 'colsample_bytree': 0.47889507164258605, 'subsample': 0.8384852560808765, 'min_child_samples': 25}


In [12]:

# best_params = {
#     "objective": "multiclass",          # Objective function for the model
#     "metric": "multi_logloss",          # Evaluation metric
#     "verbosity": -1,                    # Verbosity level (-1 for silent)
#     "boosting_type": "gbdt",            # Gradient boosting type
#     "random_state": 42,       # Random state for reproducibility
#     "num_class": 7,                     # Number of classes in the dataset
#     'learning_rate': 0.030962211546832760,  # Learning rate for gradient boosting
#     'n_estimators': 500,                # Number of boosting iterations
#     'lambda_l1': 0.009667446568254372,  # L1 regularization term
#     'lambda_l2': 0.04018641437301800,   # L2 regularization term
#     'max_depth': 10,                    # Maximum depth of the trees
#     'colsample_bytree': 0.40977129346872643,  # Fraction of features to consider for each tree
#     'subsample': 0.9535797422450176,    # Fraction of samples to consider for each boosting iteration
#     'min_child_samples': 26             # Minimum number of data needed in a leaf
# }

best_params = {'learning_rate': 0.015797659065062593, 'n_estimators': 425, 'lambda_l1': 0.008724491316001835, 'lambda_l2': 0.020209622675945973, 'max_depth': 10, 'colsample_bytree': 0.47889507164258605, 'subsample': 0.8384852560808765, 'min_child_samples': 25}

In [13]:
X = train.drop(['NObeyesdad'], axis =1)
y = train['NObeyesdad']

X_train, X_val, y_train, y_val= train_test_split(X, y, test_size = 0.2)

model = LGBMClassifier(**best_params).fit(X_train, y_train)

X_test = test

y_pred_val = model.predict(X_val)
y_pred_test = model.predict(X_test)

print(accuracy_score(y_val, y_pred_val))

[LightGBM] [Warning] lambda_l2 is set=0.020209622675945973, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.020209622675945973
[LightGBM] [Warning] lambda_l1 is set=0.008724491316001835, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.008724491316001835
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] lambda_l2 is set=0.020209622675945973, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.020209622675945973
[LightGBM] [Warning] lambda_l1 is set=0.008724491316001835, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.008724491316001835
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006902 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins

In [14]:
sample_submission['NObeyesdad'] = y_pred_test
sample_submission['id'] = sample_submission['id']
sample_submission

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III
...,...,...
13835,34593,Overweight_Level_II
13836,34594,Normal_Weight
13837,34595,Insufficient_Weight
13838,34596,Normal_Weight


In [15]:
sample_submission.to_csv('submission.csv', index = False)